In [1]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst

In [13]:
import random
import struct
import numpy
import os, os.path, shutil

**GDAL/OGR** - либы для чтения/записи геоданных

Библиотека единой абстрактной модели геоданных GDAL, первоначально была просто библиотекой для работы с растровыми геоданными, в то время как библиотека 0GR предназначалась для работы с векторными данными.

Теперь либы объединены. Стандартная версия gdal позволяет читать в 100 форматах, а писать в 71. ogr читает вы 42 форматах, пишет в 39.

Для чтения и записи данных в растровом формате библиотека GDAL исполь?
зует специальный объект dataset.

![](img/001.png)

- размер растра - общая ширина в пикселах и высота в строках. Каждая ячейка привязана географически - ей присвоены координаты в результате применения некой модели трансформации
- система координат включает картографическую проекцию, геодезический датум, единицы измерения и масштаб
- метадата содержит доп.информацию о наборе
- размер растрового канала - размер пикселов в ширину и линий в высоту для данных внутри канала.
- таблица цветности описывает как значения пикселов транислируются в цвет

каждая ячейка содержит целое число и число с плавающей точкой. gdal поддерживает отсутствие значяений

Так как у каждого растрового канала обычно имеются тысячи (и даже миллионы) ячеек, то к ним обычно обращаются не поэлементно. Вместо этого для получения доступа ко всем данным канала целиком можно воспользоваться одним из двух возможных методов:

- можно воспользоваться методом band. ReadRaster (), чтобы прочитать часть или все данные канала в двоичную строку и затем для конвертирования этой строки в массив значений использовать встроенную библиотеку struct;
- можно воспользоваться методом band.ReadArray(), чтобы прочитать часть данных канала, сохраняя значения ячеек непосредственно в объект-массив библиотеки NumPy.

#### Запись  растровых данных

Рассмотрим использование библиотеки GDAL для записи растровых данных в файл формата GeoTIFF. Для примера разделим всю поверхность Земли на 360 ячеек по горизонтали и 180 ячеек по вертикали, с тем чтобы каждая ячейка покрывала один градус широты и долготы. Для каждой ячейки сгенерируем одно случайное число между 1 и 100.

In [3]:
driver = gdal.GetDriverByName("GTiff")
dstFile = driver.Create("output/example.tiff", 
                        360, 180, 1, gdal.GDT_Int16)

Метод gdal.driver.Create() принимает следующие параметры:

- имя растрового файла; 
- нужное число ячеек по горизонтали; 
- нужное число ячеек по вертикали; 
- нужное число каналов в файле; 
- константа, задающая тип информации, хранимой в каждой ячейке. В на шем случае каждая ячейка будет содержать 16-разрядное целочисленное значение.

In [4]:
# создадим проекцию что бы спозиционировать 
# ячейки на поверхность Земли
# Вспользуемся датумом WGS84
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS("WGS84")
dstFile.SetProjection(spatialReference.ExportToWkt())

0

модель трансформации данных задается списком из шести числе (матрица афинных преобразований)

Мы можем проигнорировать математику аффинных трансформаций и задать матрицу при помощи всего четырех значений: 

- позиция Х и Y верхнего левого угла верхней левой ячейки; 
- ширина каждой ячейки, измеряемая в градусах долготы; 
- высота каждой ячейки, измеряемая в градусах широты

In [5]:
# зададим модель трансформации
originX = -180
originY = 90
cellWidth = 1.0
cellHeight = 1.0

dstFile.SetGeoTransform([originX, cellWidth, 0,
                         originY, 0, -cellHeight])

0

In [6]:
# ссылка на объект gdal.Band для сохраненеия данных в растровом канале
band = dstFile.GetRasterBand(1)

In [7]:
# сгенерируем данные для растрового канала
values = []
for row in range(180):
    row_data = []
    for col in range(360):
        row_data.append(random.randint(1, 100))
    values.append(row_data)

In [8]:
# пишем построчно в файл через стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(180):
#     scanline = struct.pack(fmt, *values[row])
#     band.WriteRaster(0, row, 360, 1, scanline)

In [9]:
# другой вариант - пишем целиком массив нампи
array = numpy.array(values, dtype=numpy.int16)
band.WriteArray(array)

In [ ]:
# нужно закерывать файл (или присваивать ему None)
del dstFile 

#### Чтение растровых данных

In [10]:
srcFile = gdal.Open("output/example.tiff")
band = srcFile.GetRasterBand(1)

In [11]:
# чтение для стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(band.YSize):
#     scanline = band.ReadRaster(0, row, band.XSize, 1,
#                                band.XSize, 1,
#                                band.DataType)
#     row_data = struct.unpack(fmt, scanline)
#     print(row_data)

In [12]:
# чтение для numpy
values = band.ReadAsArray()
for row in range(band.XSize):
    print(values[row])

[ 68  38  69  44  38  71  81  85  24  55  41   7  96  59  66  47  23  47
  19   8  60  62  17  99  99  63  42  53  62  78  63   7  76  88  48  98
  95  44  20  54  46  95  74  38  35  35  10  38  72  48  24  77  55  33
  31  75  47  40  35  65  75  91  80  41  75  93  67  99  71  58  19  49
   2  58  13  54  56  86  26   5  60   3   5  82   3  39  32  13  99  48
  97  34  69   4  55  51  44 100  93  61  34  55  80  38  87  85  75  94
  86  78  12  84  73  20  65  98  87  80  69  90  59  26  85  58  36  64
  86  86  56  37  90  52  11   8  27  38  36  80  77  82  10  72  24  65
  21  47  41  42  16  82  75  41  13  31  77  90  65  59  28  50  30  33
   8  49  16  60  94  60  35   4   1   7  33  34  92  21  82  34  55  26
  55  80  10  50   2  28  27  42  78  65   8   2  22  69  53  37  36  64
  13  53  95  27  86  81  80  61  55  11  41  96  76  81  98  54  16  47
 100  98  99  90  72  27  70 100  70  46  77  39  31  49  13  51  43  56
  72  25   1   1  93  86  76   2   7  28  85  75  9

IndexError: index 180 is out of bounds for axis 0 with size 180

Структура данных в растре

![](img/002.png)

#### Запись векторных данных

Сгенерируем набор произвольных геометрий точки и сохраним их в файле фигур вместе с некоторой атрибутивной информацией.

In [14]:
if os.path.exists("output/test-shapefile"):
    shutil.rmtree("output/test-shapefile")
os.mkdir("output/test-shapefile")

In [15]:
# зададим драйвер
driver = ogr.GetDriverByName("ESRI Shapefile")
path = os.path.join("output/test-shapefile", "shapefile.shp")
datasource = driver.CreateDataSource(path)

In [16]:
# определим слой
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS('WGS84')

0

In [17]:
# создадим слой
layer = datasource.CreateLayer("layer", spatialReference)

In [18]:
# добавим в слой типы данных и атрибуты, 
# которые мы планируем там хранить
field = ogr.FieldDefn("ID", ogr.OFTInteger)
field.SetWidth(4)
layer.CreateField(field)

field = ogr.FieldDefn("NAME", ogr.OFTString)
field.SetWidth(20)
layer.CreateField(field)

0

In [19]:
# добавим немного данных в слой
for i in range(100):
    id_ = 1000 + i
    lat  = random.uniform(-90, +90)
    long = random.uniform(-180, +180)
    name = "point-{}".format(i)

    # тут определим геометрию точки
    wkt = "POINT({} {})".format(long, lat)    
    geometry = ogr.CreateGeometryFromWkt(wkt)

    # для каждой точки создадим объект ogr.Feature
    # и сохраним в нем все данные
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(geometry)
    feature.SetField("ID", id_)
    feature.SetField("NAME", name)

    # разместим объект в слове
    layer.CreateFeature(feature)

In [20]:
# осовобождаем ресурсы
feature.Destroy()
datasource.Destroy()
del datasource

#### Чтение векторых ф-лов

In [21]:
shapefile = ogr.Open("output/test-shapefile/shapefile.shp")
layer = shapefile.GetLayer(0)

for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    id_ = feature.GetField("ID")
    name = feature.GetField("NAME")
    geometry = feature.GetGeometryRef()
    print(i, id_, name, geometry.GetX(), geometry.GetY())

0 1000 point-0 73.34734078394251 88.910262553596
1 1001 point-1 -45.337300163425 -67.19578043147487
2 1002 point-2 47.9614018760702 21.69280384943184
3 1003 point-3 122.11058305648328 45.78631766291397
4 1004 point-4 70.96383955646343 0.6464184238696902
5 1005 point-5 -1.1883995546365327 -58.54702203463691
6 1006 point-6 -169.21272366788136 18.290315504142256
7 1007 point-7 52.86104548436191 -85.6429220623914
8 1008 point-8 -81.13546836276565 63.868013710193935
9 1009 point-9 -39.145153595689266 21.110246917470022
10 1010 point-10 -96.67100944333053 -35.750558861092316
11 1011 point-11 28.77737549154861 -86.00328016321423
12 1012 point-12 -130.8194881905075 29.96152439960072
13 1013 point-13 -86.71561267041966 -69.00974692875758
14 1014 point-14 129.4036350352028 -73.13260528049513
15 1015 point-15 95.07684358421733 32.46261056608765
16 1016 point-16 -93.71502312553964 -76.71586678019767
17 1017 point-17 -17.03203394392625 49.57126972669596
18 1018 point-18 100.80056203589515 40.935606

In [22]:
import pyproj
print(pyproj.__version__)

2.6.1.post1
